In [2]:
import numpy as np
import copy
import json
import re
from fasttext import FastVector
import pandas as pd
import tensorflow as tf
import pickle
from sklearn.datasets import load_linnerud
from sklearn.model_selection import train_test_split

In [3]:
en_dictionary = FastVector(vector_file='/home/basavaraj/Downloads/wiki.en.vec')

reading word vectors from /home/basavaraj/Downloads/wiki.en.vec


In [4]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [13]:
import re
X_words = []
Y_words = []
inp = []
out = []
i=0
err=0
with open('./wiktionary.csv','r') as f:
    line = f.readlines()
    while i<len(line):
        try:
            words = [line[i+4].strip(),line[i].strip(),line[i+2].strip()]
            
        except:
            break
        i+=8
        try:
            a = list(en_dictionary[words[1].lower()])
            b = list(en_dictionary[words[2].lower()])
            c = list(en_dictionary[words[0].lower()])
            X_words.append((words[1].lower(),words[2].lower()))
            Y_words.append(words[0].lower())
            inp.append(a+b)
            out.append(c)
            
        except Exception as e:
            err+=1
                

In [14]:
print len(inp),len(out),len(X),len(Y),err

316 316 316 316 85


In [26]:
X, y = inp,out
X_train, X_test, y_train, y_test = X[0:250],X[250:],y[0:250],y[250:]

In [27]:
learning_rate = 0.0005
epochs = 300
batch_size = 1

x = tf.placeholder(tf.float32, [None, 600], name='x')  # 3 features
y = tf.placeholder(tf.float32, [None, 300], name='y')  # 3 outputs

# hidden layer 1
W1 = tf.Variable(tf.truncated_normal([600, 450], stddev=0.03), name='W1')
b1 = tf.Variable(tf.truncated_normal([450]), name='b1')

# hidden layer 2
W2 = tf.Variable(tf.truncated_normal([450, 450], stddev=0.03), name='W2')
b2 = tf.Variable(tf.truncated_normal([450]), name='b2')

# hidden layer 3
W3 = tf.Variable(tf.truncated_normal([450, 450], stddev=0.03), name='W3')
b3 = tf.Variable(tf.truncated_normal([450]), name='b3')

# hidden layer 2
W4 = tf.Variable(tf.truncated_normal([450, 450], stddev=0.03), name='W4')
b4 = tf.Variable(tf.truncated_normal([450]), name='b4')

# hidden layer 2
W5 = tf.Variable(tf.truncated_normal([450,300], stddev=0.03), name='W5')
b5 = tf.Variable(tf.truncated_normal([300]), name='b5')

######################## Activations, outputs ######################
# output hidden layer 1
hidden_out = tf.nn.softplus(tf.add(tf.matmul(x, W1), b1))

hidden_out1 = tf.nn.softplus(tf.add(tf.matmul(hidden_out, W2), b2))
hidden_out2 = tf.nn.softplus(tf.add(tf.matmul(hidden_out1, W3), b3))
hidden_out3 = tf.nn.softplus(tf.add(tf.matmul(hidden_out2, W4), b4))

# total output
y_ = tf.nn.softplus(tf.add(tf.matmul(hidden_out3, W5), b5))

####################### Loss Function  #########################
mse = tf.losses.mean_squared_error(y, y_)

####################### Optimizer      #########################
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

###################### Initialize, Accuracy and Run #################
# initialize variables
init_op = tf.global_variables_initializer()

# accuracy for the test set
accuracy = tf.reduce_mean(tf.square(tf.subtract(y, y_)))  # or could use tf.losses.mean_squared_error

In [28]:
with tf.Session() as sess:
    sess.run(init_op)
    total_batch = int(len(y_train) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = X_train[i * batch_size:min(i * batch_size + batch_size, len(X_train))], \
                             y_train[i * batch_size:min(i * batch_size + batch_size, len(y_train))]
            _, c = sess.run([optimizer, mse], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        if epoch % 10 == 0:
            print 'Epoch:', (epoch + 1), 'cost =', '{:.3f}'.format(avg_cost)
    print sess.run(mse, feed_dict={x: X_test, y: y_test})

Epoch: 1 cost = 0.890
Epoch: 11 cost = 0.165
Epoch: 21 cost = 0.098
Epoch: 31 cost = 0.079
Epoch: 41 cost = 0.071
Epoch: 51 cost = 0.067
Epoch: 61 cost = 0.065
Epoch: 71 cost = 0.063
Epoch: 81 cost = 0.062
Epoch: 91 cost = 0.061
Epoch: 101 cost = 0.061
Epoch: 111 cost = 0.060
Epoch: 121 cost = 0.060
Epoch: 131 cost = 0.059
Epoch: 141 cost = 0.059
Epoch: 151 cost = 0.059
Epoch: 161 cost = 0.059
Epoch: 171 cost = 0.059
Epoch: 181 cost = 0.058
Epoch: 191 cost = 0.058
Epoch: 201 cost = 0.058
Epoch: 211 cost = 0.058
Epoch: 221 cost = 0.058
Epoch: 231 cost = 0.058
Epoch: 241 cost = 0.058
Epoch: 251 cost = 0.058
Epoch: 261 cost = 0.058
Epoch: 271 cost = 0.058
Epoch: 281 cost = 0.058
Epoch: 291 cost = 0.058
0.0617567


In [29]:
words = []
true_out = []
for j in range(len(X_test)):
    for i in range(len(inp)):
        if cmp(inp[i],X_test[j])==0:
            words.append(X_words[i])
            true_out.append(Y[i])

In [30]:
print words[0],true_out[0]

('sound', 'landscape') soundscape


In [31]:
for k1 in range(len(X_test)):
    ans = 0 
    best = '-'
#     print words[k1][0],words[k1][1]
    for i in range(len(words[k1][0])):
        for j in range(len(words[k1][1])):
            str1 = words[k1][0][0:i]+words[k1][1][j:]
            try:
                k = cosine(en_dictionary[str1],y_test[k1]) 
                if k>ans:
                    best = str1
                    ans = k
#                 print true_out[k1],str1,k 
            except:
                er=0
    print true_out[k1],best
#     print '\n'

soundscape soundscape
spamalot camelot
spamdexing indexing
spanglish spanglish
spife spife
sporgery sporgery
spork spork
squiggle squiggle
squircle squircle
starchitect starchitect
staycation vacation
streetball streetball
strunk strunk
swaption swaption
swenglish swenglish
taglish taglish
tangelo tangelo
tanglish tanglish
tanorexia tanorexia
taxachusetts massachusetts
tehrangeles tehrangeles
telematics telematics
televangelist televangelist
thermistor thermistor
threepeat threepeat
tigon tigon
tofurkey tofurkey
tomacco tomacco
toonie toonie
transceiver transceiver
transistor transistor
transponder transponder
travelocity travelocity
treap treap
triticale triticale
troopergate troopergate
twincest twincest
urinalysis urinalysis
verizon verizon
vermonster vermonster
vitamer vitamer
vitamin vitamine
vlog vlog
voxel voxel
wallaroo wallaroo
waluigi waluigi
wario wario
webcast broadcast
wenglish wenglish
wexis wexis
wholphin wholphin
wigger wigger
wikiality eality
wikipedia wikpedia
wikique